# Dataset Generation (CP2K)

This is a example workflow using CP2K as to generate the training data in sections 1 to 5. The alternative methods, using Quantum Espresso, can be found [here](quantum_espresso.ipynb).

## Assumed directory structure

```
example_directory
├── active_learning
│   └── simulation.lammps
├── cp2k_input
│   └── example_template.inp
├── cp2k_output
├── lammps
│   └── template.lmp
├── n2p2
│   └── input.nn.template
├── scripts
│   ├── example.ipynb
│   └── template.sh
├── xyz
└── example_trajectory.history
```

In [1]:
from cc_hdnnp.data import Data
from cc_hdnnp.structure import AllSpecies, AllStructures, Species, Structure

# Create objects for all elements in the structure
H = Species(symbol='H', atomic_number=1, mass=1.00794)
C = Species(symbol='C', atomic_number=6, mass=12.011)
O = Species(symbol='O', atomic_number=8, mass=15.9994)

# Define a name for the Structure which has the above constituent elements
# Information used for active learning, such as the energy and force tolerances is also defined here
all_species = AllSpecies(H, C, O)
structure = Structure(name='mcresol', all_species=all_species, delta_E=1e-4, delta_F=1e-2)
all_structures = AllStructures(structure)

# Directories and constant file locations
main_directory = '..'
n2p2_bin = '/path/to/n2p2/bin'
lammps_executable = '/path/to/lammps/build/lmp_mpi'
basis_set = "/path/to/cp2k/data/BASIS_MOLOPT"
potential = "/path/to/cp2k/data/GTH_POTENTIALS"

d = Data(
    structures=all_structures,
    main_directory=main_directory,
    n2p2_bin=n2p2_bin,
    lammps_executable=lammps_executable
)

## 1. Generate atomic configurations
There are no utility scripts for the generation of configurations, however a full trajectory can be converted into individual frames by:

In [ ]:
d.read_trajectory(file_trajectory='example_trajectory.history')
d.write_xyz(file_xyz='xyz/{}.xyz')

## 2. Write CP2K
Both batch scripts and input files for CP2K can be generated by: 

In [ ]:
d.write_cp2k(
    file_batch='scripts/cp2k_batch_{}.bash',
    file_input='cp2k_input/example_{}.inp',
    file_xyz='xyz/{}.xyz',
    n_config=1,
    cutoff=(400, 600, 800),
    relcutoff=(40, 60, 80),
)

In this case, we have generated 9 input files and 9 batch scripts by specifying 3 values for `cutoff` and `relcutoff`. These can then be used to determine the best values for these settings (that balances accuracy with time taken).

## 3. Run CP2K
The previous step should output `bash ../scripts/all.bash`. This bash script will submit all the batch scripts which will submit Slurm jobs for all 9 cases.

In [ ]:
!bash ../scripts/all.bash


## 4. Choose (rel)cutoff
To extract the useful information from the CP2K output, the following function will print a table comparing energy, time taken and grid allocation:

In [ ]:
d.print_cp2k_table(n_config=1, cutoff=(400, 600, 800), relcutoff=(40, 60, 80))

Once the best value is chosen, to run CP2K with more frames (larger `n_config`) repeat steps 2. and 3. but with different arguments:

In [ ]:
d.write_cp2k(
    structure_name="mcresol",
    basis_set=basis_set,
    potential=potential,
    file_batch='scripts/{}.sh',
    file_input='cp2k_input/example_{}.inp',
    file_xyz='xyz/{}.xyz',
    n_config=101,
    cutoff=600,
    relcutoff=60)

In [ ]:
!bash ../scripts/all.bash

## 5. Write N2P2
Once force and energy values are obtained from CP2K, these can be written to the N2P2 data format. The structure name should match one of the structures in `all_structures`:

In [ ]:
d.write_n2p2_data(
    structure_name="mcresol",
    file_cp2k_out='cp2k_output/example_n_{}_cutoff_600_relcutoff_60.log',
    file_cp2k_forces='cp2k_output/example_n_{}_cutoff_600_relcutoff_60-forces-1_0.xyz',
    file_xyz='xyz/{}.xyz',
    file_n2p2_input='input.data',
    n_config=101)

At this point, the N2P2 training commence following TODO.